In [ ]:
pip install ultralytics

In [ ]:
pip install -U albumentations

In [27]:
pip install pyyaml

Note: you may need to restart the kernel to use updated packages.


In [ ]:
split_dataset(leaf_path, output_dir)

In [8]:
import os
os.environ['WANDB_DISABLED'] = 'true'
from ultralytics import YOLO
import cv2
import albumentations as A
from tqdm import tqdm
import torch
import torchvision
from torchvision import transforms
from sklearn.metrics import classification_report
from PIL import Image
from pathlib import Path
import yaml
import torchvision.models as models

C:\Users\faith\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated and will be removed in a future release
  "class": algorithms.Blowfish,


In [51]:
leaf_path = "E:\Snapfolia - CS\Samples\Leaf_Samples"
output_dir = "SPLITTED_DATA_YOLO_9_Samples"

In [52]:
def split_dataset(root_dir, output_dir, test_ratio=0.1, val_ratio=0.2):
    # Creating output directories
    train_dir = os.path.join(output_dir, 'train')
    val_dir = os.path.join(output_dir, 'val')
    test_dir = os.path.join(output_dir, 'test')

    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(val_dir, exist_ok=True)
    os.makedirs(test_dir, exist_ok=True)

    class_dirs = [d for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))]

    for class_dir in class_dirs:
        class_path = os.path.join(root_dir, class_dir)
        images = [f for f in os.listdir(class_path) if os.path.isfile(os.path.join(class_path, f))]
        random.shuffle(images)

        num_total = len(images)
        num_test = int(num_total * test_ratio)
        num_val = int(num_total * val_ratio)
        num_train = num_total - num_val - num_test

        test_images = images[:num_test]
        val_images = images[num_test:num_test+num_val]
        train_images = images[num_test+num_val:]

        os.makedirs(os.path.join(test_dir, class_dir), exist_ok=True)
        for img in test_images:
            shutil.copy(os.path.join(class_path, img), os.path.join(test_dir, class_dir, img))

        os.makedirs(os.path.join(val_dir, class_dir), exist_ok=True)
        for img in val_images:
            shutil.copy(os.path.join(class_path, img), os.path.join(val_dir, class_dir, img))

        os.makedirs(os.path.join(train_dir, class_dir), exist_ok=True)
        for img in train_images:
            shutil.copy(os.path.join(class_path, img), os.path.join(train_dir, class_dir, img))

    print("Dataset split completed.")

In [53]:
dataset_path = './dataset.yaml'

In [54]:
# Check if the dataset.yaml exists
if not os.path.exists(dataset_path):
    raise FileNotFoundError(f"dataset.yaml not found at {dataset_path}")


In [55]:
# Print the working directory to ensure it's correct
print("Current Working Directory:", os.getcwd())

Current Working Directory: E:\Snapfolia - CS\YOLO v8


In [62]:
model = YOLO('yolov8n-cls.pt') 

In [64]:
model.train(data='/Snapfolia - CS/YOLO v8/datasets/SPLITTED_DATA_YOLO_9_Samples', epochs=10, imgsz=640)

Ultralytics YOLOv8.2.48  Python-3.11.7 torch-2.3.1 CUDA:0 (NVIDIA GeForce RTX 3060 Ti, 8192MiB)
engine\trainer: task=classify, mode=train, model=yolov8n-cls.pt, data=/Snapfolia - CS/YOLO v8/datasets/SPLITTED_DATA_YOLO_9_Samples, epochs=10, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train33, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save

train: Scanning E:\Snapfolia - CS\YOLO v8\datasets\SPLITTED_DATA_YOLO_9_Samples\train... 3130 images, 0 corrupt: 100%|█
val: Scanning E:\Snapfolia - CS\YOLO v8\datasets\SPLITTED_DATA_YOLO_9_Samples\val... 894 images, 0 corrupt: 100%|██████


optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000714, momentum=0.9) with parameter groups 26 weight(decay=0.0), 27 weight(decay=0.0005), 27 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs\classify\train33
Starting training for 10 epochs...

      Epoch    GPU_mem       loss  Instances       Size


               classes   top1_acc   top5_acc: 100%|██████████| 28/28 [00:24<00:00,  1.17it/s]

                   all        0.9          1


...


10 epochs completed in 0.428 hours.
Optimizer stripped from runs\classify\train33\weights\last.pt, 3.0MB
Optimizer stripped from runs\classify\train33\weights\best.pt, 3.0MB

Validating runs\classify\train33\weights\best.pt...
Ultralytics YOLOv8.2.48  Python-3.11.7 torch-2.3.1 CUDA:0 (NVIDIA GeForce RTX 3060 Ti, 8192MiB)
YOLOv8n-cls summary (fused): 73 layers, 1446409 parameters, 0 gradients, 3.3 GFLOPs
train: E:\Snapfolia - CS\YOLO v8\datasets\SPLITTED_DATA_YOLO_9_Samples\train... found 3130 images in 9 classes  
val: E:\Snapfolia - CS\YOLO v8\datasets\SPLITTED_DATA_YOLO_9_Samples\val... found 894 images in 9 classes  
test: E:\Snapfolia - CS\YOLO v8\datasets\SPLITTED_DATA_YOLO_9_Samples\test... found 447 images in 9 classes  


               classes   top1_acc   top5_acc: 100%|██████████| 28/28 [00:24<00:00,  1.13it/s]


                   all       0.99          1
Speed: 1.1ms preprocess, 1.5ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs\classify\train33
Results saved to runs\classify\train33


lr/pg0,▃▆█▇▆▅▄▃▂▁
lr/pg1,▃▆█▇▆▅▄▃▂▁
lr/pg2,▃▆█▇▆▅▄▃▂▁
metrics/accuracy_top1,▁▅▅▇██████
metrics/accuracy_top5,▁▁▁▁▁▁▁▁▁▁
model/GFLOPs,▁
model/parameters,▁
model/speed_PyTorch(ms),▁
train/loss,█▃▂▁▁▁▁▁▁▁
val/loss,█▃▃▂▁▁▁▁▁▁
lr/pg0,8e-05


ultralytics.utils.metrics.ClassifyMetrics object with attributes:

confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000002B9341CEC10>
curves: []
curves_results: []
fitness: 0.994966447353363
keys: ['metrics/accuracy_top1', 'metrics/accuracy_top5']
results_dict: {'metrics/accuracy_top1': 0.9899328947067261, 'metrics/accuracy_top5': 1.0, 'fitness': 0.994966447353363}
save_dir: WindowsPath('runs/classify/train33')
speed: {'preprocess': 1.1027542003315833, 'inference': 1.5184999312330412, 'loss': 0.0, 'postprocess': 0.0}
task: 'classify'
top1: 0.9899328947067261
top5: 1.0

In [9]:
from ultralytics import YOLO

import numpy as np


model = YOLO("../../../MODELS/YOLO v8/runs/classify/train33/weights/last.pt")  # load a custom model

if torch.cuda.is_available():
    print("cuda is available")
else:
    print("cuda is not available")

predict = model(r'E:\Snapfolia - CS\MODELS\YOLO v8\datasets\test\Balete\Balete (197).jpg')  # predict on an image

names_dict = predict[0].names

probs = predict[0].probs.data.tolist()

print(names_dict)
print(probs)

results = names_dict[np.argmax(probs)]    
print(results)

cuda is available

image 1/1 E:\Snapfolia - CS\MODELS\YOLO v8\datasets\test\Balete\Balete (197).jpg: 640x640 Balete 1.00, Guyabano 0.00, Apitong 0.00, Kamagong 0.00, Langka 0.00, 0.0ms
Speed: 83.5ms preprocess, 0.0ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)
{0: 'Apitong', 1: 'Balete', 2: 'Bayabas', 3: 'Guyabano', 4: 'Kamagong', 5: 'Langka', 6: 'Mahogany', 7: 'Mangga', 8: 'Palo Maria'}
[2.932660114751684e-09, 1.0, 6.407280088438494e-12, 1.1804381472302339e-08, 2.261882903198398e-09, 2.62995125677179e-10, 1.3325726806734384e-11, 3.940153758819065e-11, 1.168740104695587e-10]
Balete


In [80]:
from sklearn.metrics import classification_report
import os
from ultralytics import YOLO
import numpy as np
from PIL import Image
from torchvision import transforms

def generate_classification_report(run_dir, data_dir, classes):
    # Load the model
    model = YOLO(f'{run_dir}/weights/best.pt')
    
    # Prepare the test dataset
    test_dir = os.path.join(data_dir, 'test')
    test_images = []
    test_labels = []
    for class_id, class_name in enumerate(classes):
        class_dir = os.path.join(test_dir, class_name)
        for image_name in os.listdir(class_dir):
            image_path = os.path.join(class_dir, image_name)
            test_images.append(image_path)
            test_labels.append(class_id)
    
    # Predict and collect results
    y_true = []
    y_pred = []
    for image_path, label in zip(test_images, test_labels):
        predict = model(image_path)
        names_dict = predict[0].names
        probs = predict[0].probs.data.tolist()
        predicted_class = np.argmax(probs)
        
        y_true.append(label)
        y_pred.append(predicted_class)
    
    # Generate classification report
    report = classification_report(y_true, y_pred, target_names=classes)
    print(report)

# Define classes
classes = ['Apitong', 'Balete', 'Bayabas', 'Guyabano', 'Kamagong', 'Langka', 'Mahogany', 'Mangga', 'Palo Maria']

# Generate classification report
generate_classification_report('runs/classify/train33', 'E:/Snapfolia - CS/YOLO v8/datasets/SPLITTED_DATA_YOLO_9_Samples', classes)



image 1/1 E:\Snapfolia - CS\YOLO v8\datasets\SPLITTED_DATA_YOLO_9_Samples\test\Apitong\Apitong (339).jpg: 640x640 Apitong 0.99, Guyabano 0.00, Balete 0.00, Kamagong 0.00, Langka 0.00, 19.0ms
Speed: 70.5ms preprocess, 19.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 E:\Snapfolia - CS\YOLO v8\datasets\SPLITTED_DATA_YOLO_9_Samples\test\Apitong\Apitong (260).JPG: 640x640 Apitong 0.99, Balete 0.01, Guyabano 0.00, Kamagong 0.00, Mangga 0.00, 19.0ms
Speed: 49.6ms preprocess, 19.0ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 E:\Snapfolia - CS\YOLO v8\datasets\SPLITTED_DATA_YOLO_9_Samples\test\Apitong\Apitong (103).jpg: 640x640 Apitong 0.97, Balete 0.02, Guyabano 0.00, Langka 0.00, Kamagong 0.00, 22.1ms
Speed: 51.2ms preprocess, 22.1ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 E:\Snapfolia - CS\YOLO v8\datasets\SPLITTED_DATA_YOLO_9_Samples\test\Apitong\Apitong (56).jpg: 640x640 Apitong 1.00, Gu